## Books Recommender System using Clustering 

In [3]:
%pip install pandas numpy matplotlib seaborn scikit-learn 


Note: you may need to restart the kernel to use updated packages.


In [120]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [121]:
books = pd.read_csv('data/books.csv', sep= ";", error_bad_lines = False, encoding = 'latin-1')
users = pd.read_csv('data/users.csv', sep= ";", error_bad_lines = False, encoding = 'latin-1')
ratings = pd.read_csv('data/ratings.csv', sep= ";", error_bad_lines = False, encoding = 'latin-1') 


C:\Users\diego\AppData\Local\Temp\ipykernel_26312\1850883518.py:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  books = pd.read_csv('data/books.csv', sep= ";", error_bad_lines = False, encoding = 'latin-1')
Skipping line 6452: expected 8 fields, saw 9
Skipping line 43667: expected 8 fields, saw 10
Skipping line 51751: expected 8 fields, saw 9

Skipping line 92038: expected 8 fields, saw 9
Skipping line 104319: expected 8 fields, saw 9
Skipping line 121768: expected 8 fields, saw 9

Skipping line 144058: expected 8 fields, saw 9
Skipping line 150789: expected 8 fields, saw 9
Skipping line 157128: expected 8 fields, saw 9
Skipping line 180189: expected 8 fields, saw 9
Skipping line 185738: expected 8 fields, saw 9

Skipping line 209388: expected 8 fields, saw 9
Skipping line 220626: expected 8 fields, saw 9
Skipping line 227933: expected 8 fields, saw 11
Skipping line 228957: expected 8 fields

In [122]:
books.head(2)

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...


In [123]:
books = books[['ISBN','Book-Title', 'Book-Author', 'Year-Of-Publication', 'Publisher',
       'Image-URL-L']]

In [124]:
books.rename(columns={
    "Book-Title": "title",
    "Book-Author": "autor",
    "Year-Of-Publication": "year",
    "Publisher": "publisher",
    "Image-URL-L": "image_url"}, inplace = True)


# Lets remane some wierd columns name
users.rename(columns={"User-ID":'user_id',
                      'Location':'location',
                     "Age":'age'},inplace=True)

# Lets remane some wierd columns name
ratings.rename(columns={"User-ID":'user_id',
                      'Book-Rating':'rating'},inplace=True)

In [125]:
print(books.shape, users.shape, ratings.shape, sep='\n')

(271360, 6)
(278858, 3)
(1149780, 3)


In [126]:
ratings['user_id'].value_counts().shape

(105283,)

In [127]:
ratings['user_id'].unique().shape

(105283,)

In [180]:
# Lets store users who had at least rated more than 200 books
#x = ratings['user_id'].value_counts() > 200
x = ratings['user_id'].value_counts().between(50, 1000, inclusive=True)

C:\Users\diego\AppData\Local\Temp\ipykernel_26312\2458003562.py:3: FutureWarning: Boolean inputs to the `inclusive` argument are deprecated in favour of `both` or `neither`.
  x = ratings['user_id'].value_counts().between(50, 1000, inclusive=True)


In [181]:
x[x].shape

(782,)

In [182]:
y= x[x].index

In [183]:
y

Int64Index([129074, 252695, 197659, 211426, 265313, 187145,  56856, 127200,
            235842, 131046,
            ...
             88793, 274808,  44296,  28634,  73681,  59727, 188951, 268622,
              9856, 155916],
           dtype='int64', length=782)

In [184]:
ratings = ratings[ratings['user_id'].isin(y)]

In [185]:
ratings.head()

,user_id,ISBN,rating
1456,277427,002542730X,10
1457,277427,0026217457,0
1458,277427,003008685X,8
1459,277427,0030615321,0
1460,277427,0060002050,0


In [186]:
ratings.shape

(304983, 3)

In [187]:
# Now join ratings with books

ratings_with_books = ratings.merge(books, on='ISBN')

In [188]:
ratings_with_books.head()

,user_id,ISBN,rating,title,autor,year,publisher,image_url
0,277427,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...
1,3363,002542730X,0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...
2,13552,002542730X,0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...
3,24194,002542730X,0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...
4,25981,002542730X,0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...


In [189]:
ratings_with_books.shape

(279623, 8)

In [190]:
number_rating = ratings_with_books.groupby('title')['rating'].count().reset_index()

In [191]:
number_rating.head()

,title,rating
0,A Light in the Storm: The Civil War Diary of ...,1
1,Clifford Visita El Hospital (Clifford El Gran...,1
2,Earth Prayers From around the World: 365 Pray...,2
3,Final Fantasy Anthology: Official Strategy Gu...,2
4,Flight of Fancy: American Heiresses (Zebra Ba...,1


In [192]:
number_rating.rename(columns={'rating':'num_of_rating'},inplace=True)

In [193]:
number_rating.head()

,title,num_of_rating
0,A Light in the Storm: The Civil War Diary of ...,1
1,Clifford Visita El Hospital (Clifford El Gran...,1
2,Earth Prayers From around the World: 365 Pray...,2
3,Final Fantasy Anthology: Official Strategy Gu...,2
4,Flight of Fancy: American Heiresses (Zebra Ba...,1


In [194]:
final_rating = ratings_with_books.merge(number_rating, on='title')

In [195]:
final_rating.head()

,user_id,ISBN,rating,title,autor,year,publisher,image_url,num_of_rating
0,277427,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,61
1,3363,002542730X,0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,61
2,13552,002542730X,0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,61
3,24194,002542730X,0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,61
4,25981,002542730X,0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,61


In [196]:
final_rating.shape

(279623, 9)

In [197]:
# Lets take those books which got at least 50 rating of user

final_rating = final_rating[final_rating['num_of_rating'] >= 20]

In [198]:
final_rating.head()

,user_id,ISBN,rating,title,autor,year,publisher,image_url,num_of_rating
0,277427,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,61
1,3363,002542730X,0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,61
2,13552,002542730X,0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,61
3,24194,002542730X,0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,61
4,25981,002542730X,0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,61


In [199]:
final_rating.shape

(63447, 9)

In [200]:
# lets drop the duplicates
final_rating.drop_duplicates(['user_id','title'],inplace=True)

In [201]:
final_rating.head()

,user_id,ISBN,rating,title,autor,year,publisher,image_url,num_of_rating
0,277427,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,61
1,3363,002542730X,0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,61
2,13552,002542730X,0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,61
3,24194,002542730X,0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,61
4,25981,002542730X,0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,61


In [202]:
# Lets create a pivot table
book_pivot = final_rating.pivot_table(columns='user_id', index='title', values= 'rating')

In [203]:
book_pivot

user_id,254,2276,2766,2977,3363,3757,4017,4385,6242,6251,...,271195,271448,271705,273979,274004,274301,274808,277427,277478,277639
title,,,,,,,,,,,,,,,,,,,,,
10 Lb. Penalty,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16 Lighthouse Road,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1984,9.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1st to Die: A Novel,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010: Odyssey Two,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Year of Wonders: A Novel of the Plague,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN
You Belong To Me,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Zen and the Art of Motorcycle Maintenance: An Inquiry into Values,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [204]:
book_pivot.shape

(1608, 776)

In [205]:
book_pivot.fillna(0, inplace=True)

In [206]:
book_pivot

user_id,254,2276,2766,2977,3363,3757,4017,4385,6242,6251,...,271195,271448,271705,273979,274004,274301,274808,277427,277478,277639
title,,,,,,,,,,,,,,,,,,,,,
10 Lb. Penalty,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
16 Lighthouse Road,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1984,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1st to Die: A Novel,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2010: Odyssey Two,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Year of Wonders: A Novel of the Plague,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
You Belong To Me,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Zen and the Art of Motorcycle Maintenance: An Inquiry into Values,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Entrenamiento del Modelo

In [207]:
from scipy.sparse import csr_matrix

In [208]:
book_sparse = csr_matrix(book_pivot)

In [209]:
book_sparse

<1608x776 sparse matrix of type '<class 'numpy.float64'>'
	with 16880 stored elements in Compressed Sparse Row format>

In [210]:
type(book_sparse)

scipy.sparse._csr.csr_matrix

In [211]:
# Now import our clustering algoritm which is Nearest Neighbors this is an unsupervised ml algo
from sklearn.neighbors import NearestNeighbors
model = NearestNeighbors(algorithm= 'brute')

In [212]:
model.fit(book_sparse)

NearestNeighbors(algorithm='brute')

In [213]:
distance, suggestion = model.kneighbors(book_pivot.iloc[237,:].values.reshape(1,-1), n_neighbors = 6)

In [214]:
distance

array([[ 0.        , 27.89265136, 28.58321186, 28.70540019, 28.70540019,
        29.12043956]])

In [215]:
suggestion

array([[237, 797,  38, 538,  67,  55]], dtype=int64)

In [216]:
for i in range(len(suggestion)):
    print(book_pivot. index[suggestion[i]])


Index(['Charming Billy', 'On the Road', 'A Monk Swimming : A Memoir',
       'Hill Towns', 'AGE OF INNOCENCE (MOVIE TIE-IN)',
       'A Thousand Acres (Ballantine Reader's Circle)'],
      dtype='object', name='title')


In [217]:
book_pivot.index[3]

'1st to Die: A Novel'

In [218]:
books_name = book_pivot.index  #guardo los nombres de libros

In [219]:
# final_rating['title'].value_counts()
ids = np.where(final_rating['title'] == "Harry Potter and the Chamber of Secrets (Book 2)")[0][0]

In [220]:
final_rating.iloc[ids]['image_url']

'http://images.amazon.com/images/P/0439064872.01.LZZZZZZZ.jpg'

In [221]:
book_name = []
for book_id in suggestion:
    book_name.append(book_pivot.index[book_id])

In [222]:
book_name[0]

Index(['Charming Billy', 'On the Road', 'A Monk Swimming : A Memoir',
       'Hill Towns', 'AGE OF INNOCENCE (MOVIE TIE-IN)',
       'A Thousand Acres (Ballantine Reader's Circle)'],
      dtype='object', name='title')

In [223]:
ids_index = []
for name in book_name[0]: 
    ids = np.where(final_rating['title'] == name)[0][0]
    ids_index.append(ids)

In [224]:
for idx in ids_index:
    url = final_rating.iloc[idx]['image_url']
    print(url)

http://images.amazon.com/images/P/038533334X.01.LZZZZZZZ.jpg
http://images.amazon.com/images/P/0140042598.01.LZZZZZZZ.jpg
http://images.amazon.com/images/P/0786884142.01.LZZZZZZZ.jpg
http://images.amazon.com/images/P/0061099694.01.LZZZZZZZ.jpg
http://images.amazon.com/images/P/002026478X.01.LZZZZZZZ.jpg
http://images.amazon.com/images/P/0449907481.01.LZZZZZZZ.jpg


In [225]:
import pickle 
pickle.dump(model, open('artifacts/model.pkl', 'wb'))
pickle.dump(books_name, open('artifacts/book_name.pkl', 'wb'))
pickle.dump(final_rating, open('artifacts/final_rating.pkl', 'wb'))
pickle.dump(book_pivot, open('artifacts/book_pivot.pkl', 'wb'))

In [226]:
## Serialización con Joblib
#import joblib
#joblib.dump(model, 'artifacts/model.joblib')
#joblib.dump(books_name, 'artifacts/books_name.joblib')
#joblib.dump(final_rating, 'artifacts/final_rating.joblib')
#joblib.dump(book_pivot, 'artifacts/book_pivot.joblib')

In [227]:
#test no necesario
#np.where(book_pivot.index == "2nd Chance")[0][0]

In [228]:
def recommend_book(book_name):
    book_id = np.where(book_pivot.index == book_name)[0][0]
    distance, suggestion = model.kneighbors(book_pivot.iloc[book_id,:].values.reshape(1,-1), n_neighbors = 6)
    
    for i in range(len(suggestion)):
        books = book_pivot.index[suggestion[i]]
        for j in books:
            print(j)

In [229]:
book_name = "You Belong To Me"
recommend_book(book_name)

You Belong To Me
The Anastasia Syndrome
One Hundred Years of Solitude (Oprah's Book Club)
The Cradle Will Fall
Loves Music, Loves to Dance
Exclusive


In [113]:
!python --version


Python 3.9.12


In [ ]:
#https://www.youtube.com/watch?v=cGoAhtl-5jA&list=PLkz_y24mlSJa37r2xNDyEgt0Z4ilHtJ07&index=13&ab_channel=DSwithBappy